# SQL 작성

In [1]:
import sys
import os
import pandas as pd
from scipy import io
import cv2
from glob import glob
sys.path.append('../../src')
import DISData as DD
import json
import csv

## SQL 연결

In [2]:
doUT = DD.SQL()

host:  192.168.0.201
user:  diadmin
password:  ········


SQL connection done


# query 작성

In [14]:
query1 = (f'''CREATE TABLE IF NOT EXISTS sviro_uncertainty_box_info(
            id INT PRIMARY KEY AUTO_INCREMENT,
            img_name VARCHAR(255) NOT NULL,
            img_dir VARCHAR(255) NOT NULL,
            img_format VARCHAR(10) NOT NULL,
            img_width INT NOT NULL,
            img_height INT NOT NULL,
            color_space VARCHAR(10) NOT NULL,
            label_name VARCHAR(255) NOT NULL,
            label_dir VARCHAR(255) NOT NULL,
            label_format VARCHAR(10) NOT NULL,
            location VARCHAR(20) NOT NULL,
            label VARCHAR(100) NOT NULL,
            label_id VARCHAR(100) NOT NULL,
            box_xtl INT,
            box_ytl INT,
            box_xbr INT,
            box_ybr INT);''')

In [15]:
query2 = (f'''CREATE TABLE IF NOT EXISTS sviro_uncertainty_landmark_info(
            id INT PRIMARY KEY AUTO_INCREMENT,
            img_name VARCHAR(255) NOT NULL,
            img_dir VARCHAR(255) NOT NULL,
            img_format VARCHAR(10) NOT NULL,
            img_width INT NOT NULL,
            img_height INT NOT NULL,
            color_space VARCHAR(10) NOT NULL,
            label_name VARCHAR(255) NOT NULL,
            label_dir VARCHAR(255) NOT NULL,
            label_format VARCHAR(10) NOT NULL,
            location VARCHAR(20) NOT NULL,
            label VARCHAR(100) NOT NULL,
            label_id VARCHAR(100) NOT NULL,
            landmark VARCHAR(1000));''')

In [5]:
query1_1 = (f'''INSERT INTO sviro_uncertainty_box_info 
        (img_name, img_dir, img_format, img_width, img_height, color_space,
        label_name, label_dir, label_format,
        location, label, label_id, box_xtl, box_ytl, box_xbr, box_ybr)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''')
query2_1 = (f'''INSERT INTO sviro_uncertainty_landmark_info 
        (img_name, img_dir, img_format, img_width, img_height, color_space,
        label_name, label_dir, label_format,
        location, label, label_id, landmark)
        VALUES (%s, %s, %s, %s, %s,  %s, %s, %s, %s, %s,  %s, %s, %s)''')

In [6]:
labels = {0:'empty', 1:'baby', 2:'child', 3:'adult', 4:'stuff', 5:'babyseat', 6:'childseat'}

In [7]:
def compress_segmap(arr, n_object=3):
    return arr[:,:,0] + n_object*arr[:,:,1] + n_object*n_object*arr[:,:,2]   

In [8]:
try:
    doUT.create_table(query1)
except Exception as e:
    print(f"Error during table creation: {e}")

Error during table creation: 1050: Table 'sviro_uncertainty_box_info' already exists


In [9]:
try:
    doUT.create_table(query2)
except Exception as e:
    print(f"Error during table creation: {e}")

Error during table creation: 1050: Table 'sviro_uncertainty_landmark_info' already exists


In [10]:
import numpy as np

In [12]:
value_list1 = []
project_id = None
rm_list = []
colors = ['RGB','IR']

for color_space in colors:
    label_path = f'Z:\\sviro_uncertainty\\data\\{color_space}\\labeldata'
    img_path = f'Z:\\sviro_uncertainty\\data\\{color_space}\\rawdata'
    img_ids = os.listdir(img_path)
    for img_id in img_ids:
        img_files = os.listdir(os.path.join(img_path,img_id))  
        for cnt, img_file in enumerate(img_files):
            img = os.path.join(img_path,img_id,img_file)
            img_name, img_format = os.path.splitext(os.path.basename(img))
            img_format = img_format.replace('.','')
            img_dir = img
    
            if color_space != 'SEGMENTATION':
                label_file = img.replace('SEGMENTATION','IR').replace('raw','label').replace('.png','.csv')
                label_dir = '/'.join(os.path.dirname(label_file).split('\\')[-6:]+[os.path.basename(label_file)])
                label_name, label_format = os.path.splitext(os.path.basename(label_file))
                label_format = label_format.replace('.','')

                img_file = os.path.basename(img)
                img_name, img_format = os.path.splitext(img_file)
                img_format = img_format.replace('.','')
                
                # print(img)
                seg_dir = img.replace(f'{color_space}','SEGMENTATION')
                
                try:
                    image = cv2.imread(img)
                
                    img_width = image.shape[1]
                    img_height = image.shape[0]
            
                    seg_img = cv2.imread(seg_dir)
                except:
                    print(f"seg_error : {img}")
                    continue

                arr = np.array(seg_img)

                digitized = np.digitize(arr, bins=[-1,190,256])
                seg_name, seg_format = os.path.splitext(os.path.basename(seg_dir))
                seg_obj = list(map(int,seg_name.split('_')[-3:]))
                i = len(seg_obj) - seg_obj.count(0)
                segmap = compress_segmap(digitized, n_object=i)

                ny, nx = segmap.shape
                unique_keys = np.unique(segmap)

                centers = []
                bboxs = []

                for unik in unique_keys:
                    tmp = np.zeros_like(segmap)
                    tmp[np.where(segmap == unik)] = unik

                    xsum = np.sum(tmp, axis=0)
                    ysum = np.sum(tmp, axis=1)

                    ytl = np.argmax(ysum > 0)
                    ybr = len(ysum) - np.argmax(ysum[::-1] > 0)
                    # print(ytl, ybr)

                    xtl = np.argmax(xsum > 0)
                    xbr = len(xsum) - np.argmax(xsum[::-1] > 0)
                    # print(xtl, xbr)
                    if xtl == 0 and xbr == nx:
                        continue

                    xcen = 0.5 * (xtl + xbr)
                    centers.append(xcen)

                    bboxs.append([xtl, ytl, xbr, ybr])
                    # print(bboxs)

                # x 좌표 기준으로 정렬
                ind_sort = np.argsort(centers)
                # print(bboxs)
                bboxs = np.array(bboxs)[ind_sort]

                label_nums = img_name.split('_')[-3:]
                label_nums = [num for num in label_nums if num != '0']
                # print(label_nums)
                for idx, label_num in enumerate(label_nums):                    
                    box_xtl, box_ytl, box_xbr, box_ybr = map(int,bboxs[idx])
                    box_area = abs(box_xtl-box_xbr) * abs(box_ytl- box_ybr)

                    with open(label_file,'r') as f:
                        data = csv.DictReader(f)

                        for i in data:
                            location = i['location']
                            label = i['label']
                            label_id = i['label_id']
                            img_dir = img_dir.replace('Z:','DataBase')
                            label_dir = label_dir.replace('Z:','DataBase')

                            values1 = (img_name, img_dir, img_format, img_width, img_height, color_space,
                                       label_name, label_dir, label_format,
                                       location, label, label_id, box_xtl, box_ytl, box_xbr, box_ybr)
                            value_list1.append(values1)
                    # except:
                    #     rm_list.append(label_file)
                if len(value_list1) >= 500:
                    
                    print(value_list1[-1])
                    print(f'{cnt} / {len(img_files)}')
                    doUT.insert_dataset_values(query1_1, value_list1)
                    value_list1 = []
doUT.insert_dataset_values(query1_1, value_list1)

print('done')

('sharan_test_imageID_1288_GT_3_0_0', 'DataBase\\sviro_uncertainty\\data\\RGB\\rawdata\\adults\\sharan_test_imageID_1288_GT_3_0_0.png', 'png', 960, 640, 'RGB', 'sharan_test_imageID_1288_GT_3_0_0', 'DataBase/sviro_uncertainty/data/RGB/labeldata/adults/sharan_test_imageID_1288_GT_3_0_0.csv', 'csv', 'left', 'Adult', 'Thirdmass0058', 0, 181, 342, 640)
325 / 6676
insert 500 done
('sharan_test_imageID_4192_GT_0_3_3', 'DataBase\\sviro_uncertainty\\data\\RGB\\rawdata\\adults\\sharan_test_imageID_4192_GT_0_3_3.png', 'png', 960, 640, 'RGB', 'sharan_test_imageID_4192_GT_0_3_3', 'DataBase/sviro_uncertainty/data/RGB/labeldata/adults/sharan_test_imageID_4192_GT_0_3_3.csv', 'csv', 'right', 'Adult', 'Secondmass0032', 589, 183, 960, 640)
545 / 6676
insert 500 done
('sharan_test_imageID_4324_GT_3_3_3', 'DataBase\\sviro_uncertainty\\data\\RGB\\rawdata\\adults\\sharan_test_imageID_4324_GT_3_3_3.png', 'png', 960, 640, 'RGB', 'sharan_test_imageID_4324_GT_3_3_3', 'DataBase/sviro_uncertainty/data/RGB/labeldat

In [13]:
value_list2 = []
project_id = None
colors = ['RGB','IR']

for color_space in colors:
    label_path = f'Z:\\sviro_uncertainty\\data\\{color_space}\\labeldata'
    label_ids = os.listdir(label_path)
    for label_id in label_ids:
        label_files = os.listdir(os.path.join(label_path,label_id))  
        for cnt, label_file in enumerate(label_files):
            label_dir = os.path.join(label_path,label_id,label_file)
            # print(label_dir)
            img_dir = '/'.join(label_file.replace('label','raw').replace('csv','png').split('\\')[-7:])
            img_load = label_file.replace('label','raw').replace('csv','png')
            # print(img_load)
            img_file = os.path.basename(img_dir)
            img_name, img_format = os.path.splitext(img_file)
            img_format = img_format.replace('.','')
            label_name, label_format = os.path.splitext(os.path.basename(label_file))
            label_format = label_format.replace('.','')

            try:
                image = cv2.imread(img)
                
                img_width = image.shape[1]
                img_height = image.shape[0]
            except Exception as e:
                error_list.append(label_dir)
                print(label_dir, e)
                continue

            with open(label_dir,'r') as f:
                data = csv.DictReader(f)
                for idx, i in enumerate(data):
                    location = i['location']
                    label = i['label']
                    label = i['label_id']
                    landmark = i['landmark']
                    if label != 'EverydayObject':
                        img_dir = img_dir.replace('Z:','DataBase')
                        label_dir = label_dir.replace('Z:','DataBase')
                        values2 = (img_name, img_dir, img_format, img_width, img_height, color_space,
                                  label_name, label_dir, label_format,
                                  location, label, label_id,landmark)
                        value_list2.append(values2)
                        if len(value_list2) >= 300:
                            print(value_list1[-2])
                            print(f'{cnt} / {len(label_files)}')
                            doUT.insert_dataset_values(query2_1, value_list2)
                            value_list2 = []
doUT.insert_dataset_values(query2_1, value_list2)
print('done')

('sharan_test_imageID_9620_GT_4_4_4', 'DataBase\\sviro_uncertainty\\data\\IR\\rawdata\\objects\\sharan_test_imageID_9620_GT_4_4_4.png', 'png', 960, 640, 'IR', 'sharan_test_imageID_9620_GT_4_4_4', 'DataBase/sviro_uncertainty/data/IR/labeldata/objects/sharan_test_imageID_9620_GT_4_4_4.csv', 'csv', 'middle', 'EverydayObject', '037_Vray', 653, 352, 743, 513)
256 / 6676
insert 300 done
('sharan_test_imageID_9620_GT_4_4_4', 'DataBase\\sviro_uncertainty\\data\\IR\\rawdata\\objects\\sharan_test_imageID_9620_GT_4_4_4.png', 'png', 960, 640, 'IR', 'sharan_test_imageID_9620_GT_4_4_4', 'DataBase/sviro_uncertainty/data/IR/labeldata/objects/sharan_test_imageID_9620_GT_4_4_4.csv', 'csv', 'middle', 'EverydayObject', '037_Vray', 653, 352, 743, 513)
508 / 6676
insert 300 done
('sharan_test_imageID_9620_GT_4_4_4', 'DataBase\\sviro_uncertainty\\data\\IR\\rawdata\\objects\\sharan_test_imageID_9620_GT_4_4_4.png', 'png', 960, 640, 'IR', 'sharan_test_imageID_9620_GT_4_4_4', 'DataBase/sviro_uncertainty/data/IR/

In [76]:
doUT.connect_end()

SQL connection closed
